<a href="https://colab.research.google.com/github/med-fakil/ML-AMCHD-/blob/main/Optimisation%20avec%20Tuner(with%20train%20of%20best%20model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pathlib
import os

import cv2
import numpy as np
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
import datetime
from tensorflow import keras
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras import layers


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_dir = '/content/drive/MyDrive/archive.zip'
import zipfile
with zipfile.ZipFile(data_dir, 'r') as zip_ref:
    zip_ref.extractall('/content/datasets')

In [4]:
data_dir = pathlib.Path('/content/datasets/AMHCD_64/AMHCD_64')
print("Le répertoire de données : {} ".format(data_dir))

Le répertoire de données : /content/datasets/AMHCD_64/AMHCD_64 


In [5]:
#Taille des images
img_height = 32
img_width = 32

In [6]:
# train_data est un objet tensor, chaque élement est un tuple (image,label) 
train_data = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  #batch_size=1,
  )


Found 25740 files belonging to 33 classes.
Using 20592 files for training.


In [7]:
# val_data est un objet tensor, chaque élement est un tuple (image,label) 
val_data = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  #batch_size=batch_size,
  )

Found 25740 files belonging to 33 classes.
Using 5148 files for validation.


In [8]:
class_names = val_data.class_names
print("Les classes :")
print(class_names)
print("Nombre de classe : {} ".format(len(class_names)))

Les classes :
['ya', 'yab', 'yach', 'yad', 'yadd', 'yae', 'yaf', 'yag', 'yagh', 'yagw', 'yah', 'yahh', 'yaj', 'yak', 'yakw', 'yal', 'yam', 'yan', 'yaq', 'yar', 'yarr', 'yas', 'yass', 'yat', 'yatt', 'yaw', 'yax', 'yay', 'yaz', 'yazz', 'yey', 'yi', 'yu']
Nombre de classe : 33 


In [9]:
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 37.4 MB/s 
     |████████████████████████████████| 1.6 MB 49.3 MB/s 


In [10]:
import keras_tuner
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers, losses
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import MaxPool2D

In [ ]:
"""
tuner = keras_tuner.BayesianOptimization(
                        hypermodel=MyHyperModel(),
                        objective = "val_accuracy",
                        max_trials =10, #max candidates to test
                        overwrite=True,
                        directory='BO_search_dir',
                        project_name='sign_language_cnn')
"""

'\ntuner = keras_tuner.BayesianOptimization(\n                        hypermodel=MyHyperModel(),\n                        objective = "val_accuracy",\n                        max_trials =10, #max candidates to test\n                        overwrite=True,\n                        directory=\'BO_search_dir\',\n                        project_name=\'sign_language_cnn\')\n'

In [11]:
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def build_model(hp):
    #defining model
    model=Sequential()
    model.add(Rescaling(1./255))
    model.add(Conv2D(hp.Int('input_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3), padding="same", input_shape=(32, 32, 3),activation="relu"))
    
    # Number of Conv Layers is up to tuning
    for i in range( hp.Int("num_conv", min_value=1, max_value=5, step=1)) :
      model.add(Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3),
                                padding='same',
                                activation='relu'))
      
      
      model.add(MaxPool2D(pool_size=(2, 2)))
      model.add(Dropout(0.5))

    model.add(Flatten())

    for i in range(hp.Int("num_dense", min_value=1, max_value=4, step=1)) :
       model.add(Dense(hp.Choice(f'n_nodes',
                                  values=[64,128, 256, 512, 1024]),activation="relu"))
       
       
    model.add(Dense(33,activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.experimental.RMSprop(learning_rate=0.001),
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    return model

In [12]:
from kerastuner.tuners import RandomSearch
tuner = RandomSearch( build_model,
                      max_trials=2,  # how many model variations to test?
                      objective = "val_accuracy",
                      #executions_per_trial=3,  # how many trials per variation? (same model could perform differently)
                      )

<ipython-input-12-03b434b35230>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [13]:
tuner.search(x=train_data, epochs=3,
             validation_data=val_data)

Trial 2 Complete [00h 01m 19s]
val_accuracy: 0.9217171669006348

Best val_accuracy So Far: 0.9640637040138245
Total elapsed time: 00h 02m 34s


In [ ]:
print(tuner.get_best_hyperparameters()[0].values)

{'input_units': 96, 'num_conv': 2, 'conv_0_units': 96, 'num_dense': 1, 'n_nodes': 512, 'conv_1_units': 32}


In [14]:
from tensorflow.keras.utils import plot_model
# Get the optimal hyperparameters
best_hps= tuner.get_best_hyperparameters(1)[0]

# get the best model
best_model = tuner.get_best_models(1)[0]

In [17]:
nblocks = best_hps.get('num_conv')
print(f'Number of conv blocks: {nblocks}')


Number of conv blocks: 1


In [19]:
# Build the model with the optimal hyperparameters
# train the model.
model = tuner.hypermodel.build(best_hps)
model.fit(train_data, 
          validation_data=val_data, 
          epochs= 5,)
           #callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

Epoch 1/5
644/644 [==============================] - 17s 24ms/step - loss: 1.5302 - accuracy: 0.5459 - val_loss: 0.2264 - val_accuracy: 0.9227
Epoch 2/5
644/644 [==============================] - 15s 23ms/step - loss: 0.2720 - accuracy: 0.9200 - val_loss: 0.1120 - val_accuracy: 0.9646
Epoch 3/5
644/644 [==============================] - 14s 22ms/step - loss: 0.1895 - accuracy: 0.9474 - val_loss: 0.0903 - val_accuracy: 0.9709
Epoch 4/5
644/644 [==============================] - 14s 22ms/step - loss: 0.1579 - accuracy: 0.9574 - val_loss: 0.0716 - val_accuracy: 0.9802
Epoch 5/5
644/644 [==============================] - 15s 23ms/step - loss: 0.1403 - accuracy: 0.9635 - val_loss: 0.0876 - val_accuracy: 0.9792


In [18]:
# display model structure
plot_model(best_model, 'best_model.png', show_shapes=True)

# show model summary
best_model.summary()

ValueError: ignored